### Colab Activity 21.6: Hyperparameter Tuning with Keras

**Expected Time = 60 minutes**



This activity focuses on using hyperparameter tuning with the `keras` library.  There are two ways to perform a grid search with `keras`, and you will implement both. While `keras_tuner` was discussed in the lectures, here you will use the `Scikit-Learn` wrapper for keras to grid search the parameters using `GridSearchCV`.  You will implement this with the `KerasClassifier` to build some basic models on the wine dataset.  

#### Index

- [Problem 1](#-Problem-1)
- [Problem 2](#-Problem-2)
- [Problem 3](#-Problem-3)
- [Problem 4](#-Problem-4)

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.utils import to_categorical

### The Data

Below, the wine dataset is loaded, split, and scaled.  

In [2]:
wine = load_wine(as_frame=True)

In [3]:
wine.frame.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [4]:
wine.frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  targe

In [5]:
X = wine.data
y = to_categorical(wine.target)

In [6]:
X_scaled = StandardScaler().fit_transform(X)

[Back to top](#-Index)

### Problem 1

#### The Build Function



To use the `KerasClassifier` you first need to write a function that creates a `keras` model and takes in arguments for the parameters you wish to search. The pseudocode for this function is given below:

```python
def create_model(optimizer=..., neurons=..., activation=..., input_dim=...):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(neurons, activation=activation, input_shape=(input_dim,)))
    model.add(tf.keras.layers.Dense(1))  # Output layer for regression
    model.compile(optimizer=..., loss=....)
    return model
```

Your goal is to complete the definition of the `create_model` function using the arguments `optimizer = 'adam'` and `neurons=50` for `activation = 'relu'` and `input_dim = 13`. Inside the function, compile the model using the selected `optimizer` and `loss= 'mse'`.



In [8]:

tf.random.set_seed(42)
# Function to create a fully connected neural network model for SciKeras
def create_model(optimizer='adam', neurons=50, activation='relu', input_dim=13):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(neurons, activation=activation, input_shape=(input_dim,)))
    model.add(tf.keras.layers.Dense(1))  # Output layer for regression
    model.compile(optimizer=optimizer, loss='mse')
    return model


### ANSWER CHECK
create_model

<function __main__.create_model(optimizer='adam', neurons=50, activation='relu', input_dim=13)>

[Back to top](#-Index)

### Problem 2

#### Creating the `KerasRegressos` model



Now, use the `create_model` function to instantiate `KerasRegressor` as `model` with  `verbose = 2`.


In [9]:

# Keras model with SciKeras wrapper
model = KerasRegressor(model=create_model, verbose=2)



[Back to top](#-Index)

### Problem 3

#### Performing the Grid Search



Now, to perform a grid search you just need to create a dictionary named `param_grid` with the hyperparameter `'model__neurons' : [10, 50, 100]`,     `'model__activation': ['relu', 'sigmoid']`,
`'model__optimizer': ['adam', 'sgd']`, `'batch_size': [1, 10]`, and
`'epochs': [10, 20]`.  

In [10]:

tf.random.set_seed(42)
# Hyperparameters to be optimized
param_grid = {
    'model__neurons': [10, 50, 100],
    'model__activation': ['relu', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [1, 10],
    'epochs': [10, 20]
}


[Back to top](#-Index)

### Problem 4

#### Fit and Evaluate the model



Use the `GridSearchCV` function with `estimator=model`, `param_grid=param_grid`, `scoring='neg_mean_squared_error'`, `cv=3`, and `verbose=2` to search your parameters and assign the results to `grid`. Next, use function `fit` on `grid` with the training data to fit your model.

In [11]:
# GridSearchCV for hyperparameter tuning
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                   scoring='neg_mean_squared_error', cv=3, verbose=2)
grid_result = grid.fit(X_scaled, y)


# Display the best hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Fitting 3 folds for each of 48 candidates, totalling 144 fits
Epoch 1/10
118/118 - 0s - loss: 0.8815 - 155ms/epoch - 1ms/step
Epoch 2/10
118/118 - 0s - loss: 0.5221 - 46ms/epoch - 387us/step
Epoch 3/10
118/118 - 0s - loss: 0.3993 - 46ms/epoch - 386us/step
Epoch 4/10
118/118 - 0s - loss: 0.3418 - 45ms/epoch - 378us/step
Epoch 5/10
118/118 - 0s - loss: 0.3104 - 45ms/epoch - 379us/step
Epoch 6/10
118/118 - 0s - loss: 0.2919 - 44ms/epoch - 373us/step
Epoch 7/10
118/118 - 0s - loss: 0.2794 - 44ms/epoch - 371us/step
Epoch 8/10
118/118 - 0s - loss: 0.2711 - 44ms/epoch - 369us/step
Epoch 9/10
118/118 - 0s - loss: 0.2641 - 45ms/epoch - 378us/step
Epoch 10/10
118/118 - 0s - loss: 0.2598 - 44ms/epoch - 376us/step
60/60 - 0s - 47ms/epoch - 791us/step
[CV] END batch_size=1, epochs=10, model__activation=relu, model__neurons=10, model__optimizer=adam; total time=   0.7s
Epoch 1/10
119/119 - 0s - loss: 4.2447 - 143ms/epoch - 1ms/step
Epoch 2/10
119/119 - 0s - loss: 2.1537 - 51ms/epoch - 426us/step
Epo

Finally, the results are written in a dataframe.

In [13]:

# Extract and display results from GridSearchCV
results = pd.DataFrame(grid_result.cv_results_)
print(results.head())
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.612178      0.019952         0.062829        0.009713   
1       0.479934      0.006648         0.055437        0.001117   
2       0.578742      0.012192         0.053646        0.001039   
3       0.489209      0.037004         0.053922        0.000084   
4       0.568626      0.003591         0.053033        0.000264   

  param_batch_size param_epochs param_model__activation param_model__neurons  \
0                1           10                    relu                   10   
1                1           10                    relu                   10   
2                1           10                    relu                   50   
3                1           10                    relu                   50   
4                1           10                    relu                  100   

  param_model__optimizer                                             params  \
0                   adam  {'batch_siz

Because of the grading enviornment, a more exhaustive search over additional parameters is not an option.  To extend the work here should be straightforward enough, and this is a nice solution to grid searching the hyperparameters of a model.

## Summary of Exercises

This notebook demonstrates hyperparameter tuning for a Keras neural network using Scikit-Learn's GridSearchCV. The exercises guide you through:

1. **Model Creation**: Built a Keras Sequential model with configurable parameters (optimizer, neurons, activation, input dimension)
2. **Wrapper Setup**: Created a KerasRegressor instance to interface with Scikit-Learn
3. **Parameter Grid**: Defined a search space for hyperparameters including:
   - Number of neurons in hidden layer
   - Activation functions
   - Optimizers
   - Batch sizes
   - Epochs
4. **Grid Search**: Performed exhaustive search using GridSearchCV with:
   - 3-fold cross-validation
   - Negative mean squared error as scoring metric
   - Verbose output for monitoring progress

## Key Takeaways

- **Scikit-Learn Integration**: Keras models can be wrapped using KerasRegressor/KerasClassifier for compatibility with Scikit-Learn tools
- **Hyperparameter Tuning**: GridSearchCV provides a systematic way to explore optimal model configurations
- **Model Configuration**: Demonstrated how to create flexible Keras models that accept hyperparameters as arguments
- **Evaluation**: Used cross-validation to assess model performance across different parameter combinations

This approach provides a foundation for more advanced hyperparameter optimization techniques and can be extended to larger parameter spaces and more complex models.